In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('per.csv')
data.head()

,NewsID,Title,Body,Date,Time,Category,Category2
0,843656,\nوزير علوم درجمع استادان نمونه: سن بازنشستگي ...,\nوزير علوم در جمع استادان نمونه كشور گفت: از ...,\n138/5//09,\n0:9::18,\nآموزشي-,\nآموزشي
1,837144,\nگردهمايي دانش‌آموختگان موسسه آموزش عالي سوره...,\nبه گزارش سرويس صنفي آموزشي خبرگزاري دانشجويا...,\n138/5//09,\n1:4::11,\nآموزشي-,\nآموزشي
2,436862,\nنتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور...,\nنتايج آزمون دوره‌هاي فراگير مقاطع كارشناسي و...,\n138/3//07,\n1:0::03,\nآموزشي-,\nآموزشي
3,227781,\nهمايش يكروزه آسيب شناسي مفهوم روابط عمومي در...,\n,\n138/2//02,\n1:3::42,\nاجتماعي-خانواده-,\nاجتماعي
4,174187,\nوضعيت اقتصادي و ميزان تحصيلات والدين از مهمت...,\nمحمدتقي علوي يزدي، مجري اين طرح پژوهشي در اي...,\n138/1//08,\n1:1::49,\nآموزشي-,\nآموزشي


In [3]:
with open('stopwords.txt') as stopwords_file:
    stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n', '') for line in stopwords]
stopwords

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 547: character maps to <undefined>

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/masoud/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
nltk_stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(nltk_stopwords)
len(stopwords)

1495

In [6]:
import hazm
stemmer = hazm.Stemmer()

In [7]:
stemmer.stem('کتاب‌ها')

'کتاب'

In [8]:
from hazm import word_tokenize

In [9]:
dataset = pd.DataFrame(columns=('title_body', 'category'))
for index, row in data.iterrows():
    title_body = row['Title'] + ' ' + row['Body']
    title_body_tokenized = word_tokenize(title_body)
    title_body_tokenized_filtered = [w for w in title_body_tokenized if not w in stopwords]
    title_body_tokenized_filtered_stemmed = [stemmer.stem(w) for w in title_body_tokenized_filtered]
    dataset.loc[index] = {
        'title_body': ' '.join(title_body_tokenized_filtered_stemmed),
        'category': row['Category2'].replace('\n', '')
    }

In [10]:
dataset.head()

,title_body,category
0,وزير علو درجمع استاد نمونه سن بازنشستگي استاد ...,آموزشي
1,گردهمايي دانش‌آموختگ موسسه آموز عالي سوره برگز...,آموزشي
2,نتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور ن...,آموزشي
3,هماي يكروزه آسيب شناسي مفهو روابط عمومي بابلسر...,اجتماعي
4,وضعي اقتصادي ميز تحصيل والدين مهمترين عوامل مو...,آموزشي


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
vectorizer = TfidfVectorizer()
vectorizer.fit(dataset['title_body'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [13]:
X = vectorizer.transform(dataset['title_body'])

In [14]:
X

<10999x60814 sparse matrix of type '<class 'numpy.float64'>'
	with 1785147 stored elements in Compressed Sparse Row format>

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
le = LabelEncoder()
y = le.fit_transform(dataset['category'])

In [17]:
le.classes_

array(['آموزشي', 'اجتماعي', 'اقتصادي', 'بهداشتي', 'تاريخي', 'سياسي',
       'علمي', 'فرهنگي', 'فقه و حقوق', 'مذهبي', 'ورزشي'], dtype=object)

In [18]:
len(y)

10999

In [19]:
import numpy as np
print(np.unique(dataset['category']))

['آموزشي' 'اجتماعي' 'اقتصادي' 'بهداشتي' 'تاريخي' 'سياسي' 'علمي' 'فرهنگي'
 'فقه و حقوق' 'مذهبي' 'ورزشي']


In [20]:
np.shape(X)

(10999, 60814)

In [21]:
np.shape(y)

(10999,)

In [24]:
from sklearn.decomposition import PCA, KernelPCA
kpca = KernelPCA(n_components=100)

In [25]:
kpca.fit(X)

KernelPCA(alpha=1.0, coef0=1, copy_X=True, degree=3, eigen_solver='auto',
          fit_inverse_transform=False, gamma=None, kernel='linear',
          kernel_params=None, max_iter=None, n_components=1000, n_jobs=None,
          random_state=None, remove_zero_eig=False, tol=0)

In [26]:
X = kpca.transform(X)

In [28]:
np.shape(X)

(10999, 1000)

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [31]:
from sklearn import svm

In [32]:
svmc = svm.SVC()
svmc.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [33]:
svmc.score(X_test, y_test)

0.8341818181818181

In [37]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = svmc.predict(X_test)
classification_report(y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.86      0.90      0.88       259\n           1       0.65      0.66      0.65       270\n           2       0.76      0.78      0.77       256\n           3       0.86      0.91      0.89       255\n           4       0.89      0.91      0.90       240\n           5       0.77      0.74      0.75       234\n           6       0.80      0.76      0.78       265\n           7       0.85      0.82      0.84       242\n           8       0.90      0.88      0.89       254\n           9       0.94      0.94      0.94       252\n          10       0.96      0.92      0.94       223\n\n    accuracy                           0.84      2750\n   macro avg       0.84      0.84      0.84      2750\nweighted avg       0.84      0.84      0.84      2750\n'

In [38]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88       259
           1       0.65      0.66      0.65       270
           2       0.76      0.78      0.77       256
           3       0.86      0.91      0.89       255
           4       0.89      0.91      0.90       240
           5       0.77      0.74      0.75       234
           6       0.80      0.76      0.78       265
           7       0.85      0.82      0.84       242
           8       0.90      0.88      0.89       254
           9       0.94      0.94      0.94       252
          10       0.96      0.92      0.94       223

    accuracy                           0.84      2750
   macro avg       0.84      0.84      0.84      2750
weighted avg       0.84      0.84      0.84      2750



In [40]:
print(confusion_matrix(y_test, y_pred))

[[234  10   0   1   0   2   9   1   1   0   1]
 [ 14 179  21   6   8   9   7  16   5   4   1]
 [  3  26 200   0   4   5  12   3   1   2   0]
 [  1   7   4 233   0   2   8   0   0   0   0]
 [  0   4   1   0 219   9   1   2   3   1   0]
 [  2   9  17   1   7 172   5   4  14   2   1]
 [ 14   8   9  25   0   3 201   3   0   0   2]
 [  2  12   5   1   4   5   5 199   1   4   4]
 [  1   8   3   1   3  10   1   2 224   1   0]
 [  0   6   2   1   1   2   0   2   1 237   0]
 [  1   8   0   1   0   3   2   2   0   0 206]]
